## ReadGoldStandard

Script to read GoldStandard in the folder `pro_con_500`.

In [123]:
# IMPORTS
import pandas as pd
import json
import os

In [124]:
# makedirs if not exist
os.makedirs("../../etl/data/intermediate/ReadGoldStandard", exist_ok=True)

In [125]:
FILENAMES = ["annotator1.conll", "annotator2.conll", "annotator3.conll"]
RAW_DATA_FOLDER_PATH = f"../../etl/data/raw/pro_con_500"

EXPORT_PATH = "../../etl/data/intermediate/ReadGoldStandard/pro_con_500.csv"


In [126]:
x = """
neff4
c4,5
c1,51
"""

for y in x.split("\n"):
    if re.match(r"([a-z]{1}\d+?\,\d+?)", y):
        print(y)

c4,5
c1,51


In [166]:
# sents,rels={},{}
# c=0

def read_conll_file(filepath):
    """Read CoNLL formatted file and add annotations."""
    sents=[]
    anns=[]
    sent=[]
    ann=[]
    with open(filepath,"r") as f:
        for line in f:
            if line[0]=='#' or line.strip()=='' or re.match('^\d$', line):
                sents.append(sent) if sent else None
                anns.append(ann) if sent else None
                sent = []
                ann = []
                continue
            # then start counting
            if line.startswith(tuple('0123456789')):
                sent.append(line)
            elif re.match(r"([a-z]{1}\d+?\,\d+?)", line):
                ann.append(line.strip().strip(","))
    return sents, anns

            # from IPython.core.debugger import Pdb; Pdb().set_trace()

            #if c>9000: # nur fürs testen, dann weg
            #    break

In [167]:
def postprocess_sentence(sent):
    """Preprocess the sentence"""
    tokens = []
    for line in sent:
        tokens.append(line.split("\t")[1])
    # print(tokens)
    return tokens

def postprocess_ann(ann):
    """Convert the textual annotations into a structured form"""
    if ann:
        rels = []
        for rel in ann:
            try:
                rel_type = rel[0:1]
                fro_m, to = rel[1:].split(",")
                rels.append((rel_type, fro_m, to))
            except Exception as e:
                print(f"Rel: {rel}")
        return rels
    return []

In [168]:
keys = ["annotator1", "annotator2", "annotator3"]
dataset = {}

# read in all the data
for key, fname in zip(keys, FILENAMES):
    s, a = read_conll_file(f"{RAW_DATA_FOLDER_PATH}/{fname}")
    s_clean = list(map(postprocess_sentence, s))
    a_clean = list(map(postprocess_ann, a))
    dataset.update({key: [s_clean, a_clean]})

In [169]:
dataset

{'annotator1': [[['Dabei',
    'machte',
    'er',
    'sich',
    'primär',
    'für',
    'Ausgabenkürzungen',
    'stark',
    ',',
    'während',
    'er',
    'von',
    'Steuererhöhungen',
    'abriet',
    '.'],
   ['Bundesrätin',
    'Calmy-Rey',
    'ist',
    'allerdings',
    'der',
    'Einladung',
    'zum',
    'Gipfeltreffen',
    'nicht',
    'gefolgt',
    ',',
    'und',
    'einige',
    'CEO',
    'haben',
    'ihre',
    'Teilnahme',
    'ebenfalls',
    'kurzfristig',
    'abgesagt',
    '.'],
   ['Das',
    'Bundesgericht',
    'hat',
    'eine',
    'staatsrechtliche',
    'Beschwerde',
    'der',
    'SVP',
    'des',
    'Kantons',
    'Zürich',
    'abgewiesen',
    '.'],
   ['Generell',
    'ortet',
    'Zeuske',
    'denn',
    'auch',
    'die',
    'Krise',
    'Kubas',
    'zuerst',
    'in',
    'dem',
    'herrschenden',
    'absurden',
    'und',
    'ineffizienten',
    'Wirtschaftssystem',
    ',',
    'das',
    'jegliche',
    'Eigeninitiative',
 

### Merge the different annotations into a single dataset

The current strategy is to intersect all of them!

==> So our gold standard only exists of **attitude relations** where everybody agrees.

In [170]:
# abc
dataset.items()

dict_items([('annotator1', [[['Dabei', 'machte', 'er', 'sich', 'primär', 'für', 'Ausgabenkürzungen', 'stark', ',', 'während', 'er', 'von', 'Steuererhöhungen', 'abriet', '.'], ['Bundesrätin', 'Calmy-Rey', 'ist', 'allerdings', 'der', 'Einladung', 'zum', 'Gipfeltreffen', 'nicht', 'gefolgt', ',', 'und', 'einige', 'CEO', 'haben', 'ihre', 'Teilnahme', 'ebenfalls', 'kurzfristig', 'abgesagt', '.'], ['Das', 'Bundesgericht', 'hat', 'eine', 'staatsrechtliche', 'Beschwerde', 'der', 'SVP', 'des', 'Kantons', 'Zürich', 'abgewiesen', '.'], ['Generell', 'ortet', 'Zeuske', 'denn', 'auch', 'die', 'Krise', 'Kubas', 'zuerst', 'in', 'dem', 'herrschenden', 'absurden', 'und', 'ineffizienten', 'Wirtschaftssystem', ',', 'das', 'jegliche', 'Eigeninitiative', 'abwürgt', '.'], ['In', 'einer', 'Videoaufzeichnung', 'hat', 'der', 'tschetschenische', 'Rebellenführer', 'Aslan', 'Maschadow', 'eine', 'Intensivierung', 'der', 'Kämpfe', 'auf', 'fremdem', 'Territorium', 'angekündigt', 'und', 'dem', 'nächsten', 'Regierungsch

In [179]:
gold_standard = []
gs_sents = []
gs_anns = []

data=dataset["annotator1"]
sents = data[0]
anns = data[1]

for i, sent, ann in zip(range(0, len(sents)), sents,anns):
    sent = " ".join(sent)
    # print(sent)
    gs_sents.append(sent)
    print(f"Annotation length before intersection: { len(ann) }")
    ann = list(set(ann)
         .intersection(dataset["annotator2"][1][i])
         .intersection(dataset["annotator3"][1][i])
        )
    print(f"Annotation length after intersection: { len(ann) }")
    gs_anns.append(ann)
   

gold_standard=[gs_sents, gs_anns]

Annotation length before intersection: 2
Annotation length after intersection: 2
Annotation length before intersection: 4
Annotation length after intersection: 1
Annotation length before intersection: 2
Annotation length after intersection: 1
Annotation length before intersection: 2
Annotation length after intersection: 1
Annotation length before intersection: 4
Annotation length after intersection: 1
Annotation length before intersection: 2
Annotation length after intersection: 1
Annotation length before intersection: 1
Annotation length after intersection: 0
Annotation length before intersection: 1
Annotation length after intersection: 1
Annotation length before intersection: 3
Annotation length after intersection: 0
Annotation length before intersection: 2
Annotation length after intersection: 1
Annotation length before intersection: 2
Annotation length after intersection: 1
Annotation length before intersection: 0
Annotation length after intersection: 0
Annotation length before int

In [180]:
gold_standard

[['Dabei machte er sich primär für Ausgabenkürzungen stark , während er von Steuererhöhungen abriet .',
  'Bundesrätin Calmy-Rey ist allerdings der Einladung zum Gipfeltreffen nicht gefolgt , und einige CEO haben ihre Teilnahme ebenfalls kurzfristig abgesagt .',
  'Das Bundesgericht hat eine staatsrechtliche Beschwerde der SVP des Kantons Zürich abgewiesen .',
  'Generell ortet Zeuske denn auch die Krise Kubas zuerst in dem herrschenden absurden und ineffizienten Wirtschaftssystem , das jegliche Eigeninitiative abwürgt .',
  'In einer Videoaufzeichnung hat der tschetschenische Rebellenführer Aslan Maschadow eine Intensivierung der Kämpfe auf fremdem Territorium angekündigt und dem nächsten Regierungschef die sofortige Ermordung angedroht .',
  'Hofstetter wirft im Interview den Gewerkschaften vor , sie wollen die Branche anschwärzen .',
  'Dass der an der Rotmeerküste und an der Grenze zu Eritrea operierende Beja Congress ausgerechnet jetzt Unruhen anzettelt , ist kein Zufall .',
  'Da

In [281]:
# df = pd.DataFrame(sent_list).rename(columns={0: "content"})

In [282]:
# df

In [283]:
# df.to_csv(EXPORT_PATH, sep="\t")

### Generate Inferred Relations

Are all these relations inferred?

Sometimes the source is 0, which hints at a **passive construction**.

**Ignore** holderless and targetless constructions currently!!!

In [194]:
rels = []
sents_list = gold_standard[0]
rels_list = gold_standard[1]

for i, s, r in zip(range(len(sents_list)), sents_list, rels_list):
    # split sentence
    tokens = s.split(" ")
    for rel in r:
        arg1_head = tokens[int(rel[1]) - 1]
        arg2_head = tokens[int(rel[2]) - 1]
        rel_type = "pro" if rel[0]=="p" else "con"

        ### TODO => how do we extract this.
        # get the charspans of the verbs, heads and phrases
        charspan_arg1_head = s.find(arg1_head)
        charspan_arg2_head = s.find(arg2_head)
        
        verb_form_dummy=tokens[-1]
        charspan_verb_form_dummy = s.find(verb_form_dummy)

        rels.append({
                        "doc_id": i,
                        "verb_form": verb_form_dummy,
                        "verb_form_start": charspan_verb_form_dummy,
                        "verb_form_end": charspan_verb_form_dummy + len(verb_form_dummy),
                        "verb_lemma": None,
                        "arg1": None,
                        "arg1_start": None,
                        "arg1_end": None,
                        "arg1_pos": None,
                        "arg1_head": arg1_head,
                        "arg1_head_start": charspan_arg1_head,
                        "arg1_head_end": charspan_arg1_head + len(arg1_head),
                        "arg2": None,
                        "arg2_start": None,
                        "arg2_end": None,
                        "arg2_pos": None,
                        "arg2_head": arg2_head,
                        "arg2_head_start": charspan_arg2_head,
                        "arg2_head_end": charspan_arg2_head + len(arg2_head),
                        "rel_type": rel_type,
                        "pred_serial": None,
                        "full_sentence_text": s,
        })

In [197]:
rels

[{'doc_id': 0,
  'verb_form': '.',
  'verb_form_start': 97,
  'verb_form_end': 98,
  'verb_lemma': None,
  'arg1': None,
  'arg1_start': None,
  'arg1_end': None,
  'arg1_pos': None,
  'arg1_head': 'er',
  'arg1_head_start': 13,
  'arg1_head_end': 15,
  'arg2': None,
  'arg2_start': None,
  'arg2_end': None,
  'arg2_pos': None,
  'arg2_head': 'Ausgabenkürzungen',
  'arg2_head_start': 32,
  'arg2_head_end': 49,
  'rel_type': 'pro',
  'pred_serial': None,
  'full_sentence_text': 'Dabei machte er sich primär für Ausgabenkürzungen stark , während er von Steuererhöhungen abriet .'},
 {'doc_id': 0,
  'verb_form': '.',
  'verb_form_start': 97,
  'verb_form_end': 98,
  'verb_lemma': None,
  'arg1': None,
  'arg1_start': None,
  'arg1_end': None,
  'arg1_pos': None,
  'arg1_head': 'er',
  'arg1_head_start': 13,
  'arg1_head_end': 15,
  'arg2': None,
  'arg2_start': None,
  'arg2_end': None,
  'arg2_pos': None,
  'arg2_head': 'Steuererhöhungen',
  'arg2_head_start': 73,
  'arg2_head_end': 89,
  

In [198]:
df = pd.DataFrame(rels)

In [199]:
df

,doc_id,verb_form,verb_form_start,verb_form_end,verb_lemma,arg1,arg1_start,arg1_end,arg1_pos,arg1_head,...,arg2,arg2_start,arg2_end,arg2_pos,arg2_head,arg2_head_start,arg2_head_end,rel_type,pred_serial,full_sentence_text
0,0,.,97,98,None,None,None,None,None,er,...,None,None,None,None,Ausgabenkürzungen,32,49,pro,None,Dabei machte er sich primär für Ausgabenkürzun...
1,0,.,97,98,None,None,None,None,None,er,...,None,None,None,None,Steuererhöhungen,73,89,con,None,Dabei machte er sich primär für Ausgabenkürzun...
2,1,.,152,153,None,None,None,None,None,CEO,...,None,None,None,None,Teilnahme,111,120,con,None,Bundesrätin Calmy-Rey ist allerdings der Einla...
3,2,.,93,94,None,None,None,None,None,Bundesgericht,...,None,None,None,None,Beschwerde,44,54,con,None,Das Bundesgericht hat eine staatsrechtliche Be...
4,3,.,159,160,None,None,None,None,None,Zeuske,...,None,None,None,None,Wirtschaftssystem,102,119,con,None,Generell ortet Zeuske denn auch die Krise Kuba...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,482,.,138,139,None,None,None,None,None,Er,...,None,None,None,None,Vorgehen,28,36,pro,None,Er lobte das « pragmatische Vorgehen » der Reg...
186,482,.,138,139,None,None,None,None,None,Er,...,None,None,None,None,Regierung,43,52,pro,None,Er lobte das « pragmatische Vorgehen » der Reg...
187,487,.,102,103,None,None,None,None,None,Behörden,...,None,None,None,None,Vorwurf,66,73,con,None,Die Behörden selbst allerdings mussten sich am...
188,488,.,119,120,None,None,None,None,None,Rahner,...,None,None,None,None,Anthropologien,104,118,con,None,"Rahner bleibt an diesem Punkt unklar , verteid..."


In [200]:
df.to_csv(EXPORT_PATH)